## Installation and configuration¶

This notebook configures the notebooks in this tutorial to connect to an Azure Machine Learning (AML) Workspace. You can use an existing workspace or create a new one.

In [2]:
import azureml.core
from azureml.core import Workspace
from dotenv import set_key, get_key, find_dotenv
from pathlib import Path
from testing_utilities import get_auth
#from MetricsUtils.hpStatisticsCollection import statisticsCollector, CollectionEntry
#from MetricsUtils.storageutils import storageConnection

ModuleNotFoundError: No module named 'MetricsUtils'

## Prerequisites



If you have already completed the prerequisites, you can execute following command to ensure you are using correct conda environment. The output of this command should contain "tutorial_env" in the path, e.g. `/anaconda/envs/tutorial_env/bin/python`

The AML Python SDK is already installed. Let's check the AML SDK version.

In [3]:
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.57


## Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:

* An Azure subscription id
* A resource group name
* A name for your workspace
* A region for your workspace

We also require you to provide variable names that will be used to create images, deployment computes, etc. in later notebooks.

**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

Replace the values in the following cell with your information. If you would like to use service principal authentication as described [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb) make sure you provide the optional values as well.

In [4]:
# Azure resources
subscription_id = "32cf04de-62b6-46b8-b31a-863d7fd95678"
resource_group = "azuremlresourcegroup"  
workspace_name = "workspace"  
workspace_region = "eastus"  # e.g. workspace_region =  "eastus"

# Docker image and Azure Kubernetes Service (AKS) Cluster - deployment compute
image_name = (
    "kerasimage"
)  # e.g. image_name = "kerasimage (avoid underscore in names)"

#storageConnString = "YOUR_STORAGE_CONNECTION_STRING"

Create and initialize a dotenv file for storing parameters used in multiple notebooks.

In [5]:
env_path = find_dotenv()
if env_path == "":
    Path(".env").touch()
    env_path = find_dotenv()

In [6]:
set_key(env_path, "subscription_id", subscription_id)
set_key(env_path, "resource_group", resource_group)
set_key(env_path, "workspace_name", workspace_name)
set_key(env_path, "workspace_region", workspace_region)

set_key(env_path, "image_name", image_name)

(True, 'image_name', 'kerasimage')

## Create the workspace
This cell will create an AML workspace for you in a subscription, provided you have the correct permissions.
This will fail when:

1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AML!

In [10]:
from azureml.core.authentication import MsiAuthentication
from azureml.core import Workspace

msi_auth = MsiAuthentication()

In [11]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    auth=msi_auth
)

HTTPError: 

In [12]:
# import the Workspace class and check the azureml SDK version
# from azureml.core import Workspace

#statisticsCollector.startTask(CollectionEntry.AML_WORKSPACE_CREATION)
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    create_resource_group=True,
    auth=get_auth(),
    exist_ok=True,
)
#statisticsCollector.endTask(CollectionEntry.AML_WORKSPACE_CREATION)
#print(statisticsCollector.getEntry(CollectionEntry.AML_WORKSPACE_CREATION))
# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Below we will reload it just to make sure that everything is working.

In [13]:
# load workspace configuratio from ./aml_config/config.json file.
ws = Workspace.from_config(auth=get_auth())
ws.get_details()

{'id': '/subscriptions/32cf04de-62b6-46b8-b31a-863d7fd95678/resourceGroups/azuremlresourcegroup/providers/Microsoft.MachineLearningServices/workspaces/workspace',
 'name': 'workspace',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'workspaceid': '6186d85d-5913-4781-b1e1-363d0dd52c07',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-04-20T09:33:33.4109093+00:00',
 'keyVault': '/subscriptions/32cf04de-62b6-46b8-b31a-863d7fd95678/resourcegroups/azuremlresourcegroup/providers/microsoft.keyvault/vaults/workspace6125817589',
 'applicationInsights': '/subscriptions/32cf04de-62b6-46b8-b31a-863d7fd95678/resourcegroups/azuremlresourcegroup/providers/microsoft.insights/components/workspace3992835842',
 'identityPrincipalId': 'd34bcd74-81d3-4111-ad7a-5d650e1d2cff',
 'identityTenantId': '18b1cb95-a3f5-470d-821c-afa7ba83b1c6',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/32cf04de-62b6-46b8-b31a-863d7fd95678/reso

Get a connection string to the workspace's storage to use to save the statistics.

In [ ]:
if storageConnString == "YOUR_STORAGE_CONNECTION_STRING":
    stgAcctName = ws.get_details()['storageAccount'].split('/')[-1]
    storageConnString = storageConnection.getConnectionStringWithAzCredentials(resource_group, stgAcctName)
set_key(env_path, "storage_conn_string", storageConnString)
print("storage_conn_string: {}".format(storageConnString))

Save the statistics collected so far.

In [ ]:
statisticsCollector.uploadContent(storageConnString)

In this notebook, we created a ".env" file to save and reuse the variables needed cross all the notebooks. We also created a new Azure resource group with name <YOUR\_RESOURCE\_GROUP>, where an AML workspace and a few other Azure resources are created. We can now move on to the next notebook [developing the model](01_DevelopModel.ipynb).